In [2]:
import pandas as pd
import numpy as np

In [3]:
house = pd.read_csv("house.csv")
item = pd.read_csv("item.csv")
house_item = pd.read_csv("hi_interaction.csv")
card_1by1 = pd.read_csv("house2card_1by1.csv")
house_card = pd.read_csv("hc_interaction.csv")
# card = pd.read_csv("card.csv")

# house와 card 1:1 매칭된 새로운 house 생성

In [4]:
hc_temp = {i:j for i, j in zip(house_card.card.values, house_card.house.values)}
card_1by1.columns = ['card', 'image_space', 'img_url']
card_1by1['house'] = card_1by1.card.map(hc_temp)
card_temp_space = {i:j for i, j in zip(card_1by1.house.values, card_1by1.image_space.values)}
card_temp = {i:j for i, j in zip(card_1by1.house.values, card_1by1.img_url.values)}
house['card_space'] = house.house.map(card_temp_space)
house['card_img_url'] = house.house.map(card_temp)

house.조회 = house.조회.apply(lambda x: x.replace(',', ''))
house.스크랩 = house.스크랩.apply(lambda x: x.replace(',', ''))
house.좋아요 = house.좋아요.apply(lambda x: x.replace(',', ''))

house.조회 = pd.to_numeric(house.조회)
house.스크랩 = pd.to_numeric(house.스크랩)
house.좋아요 = pd.to_numeric(house.좋아요)

In [5]:
house = house.rename(columns={"공간":"space", "평수":"size", "작업":"work", "분야":"category", "가족형태":"family", "지역":"region", "스타일":"style", "기간":"duration","예산":"budget","세부공사":"detail","좋아요":"prefer","스크랩":"scrab","댓글":"comment","조회":"views"})

# item.csv 전처리

In [6]:
item = item.fillna(np.nan)

## predict_price 예상가격만 남기기

In [7]:
predict_price_list = []

import re
p = re.compile('.+(?=사)')
for i in item.predict_price.index:
    m = p.findall(str(item.predict_price.iloc[i]))
    if len(m) == 0:
        m.append("예상가정보없음")
    predict_price_list.append(m[0][3:])

In [8]:
item.predict_price = predict_price_list

## 별점 숫자만 남기기

In [9]:
item.rating = item.rating.apply(lambda x: str(x)[3:-1])

In [10]:
item.rating = item.rating.apply(lambda x: np.nan if x == '' else x)

## review 리뷰 개수만 남기기

In [11]:
item.review = item.review.fillna("리뷰 쓰기첫 리뷰 두 배 적립")

In [12]:
review_list = []

import re
p = re.compile('[(][0-9]*[)]')
for idx, review in enumerate(item.review):
    m = p.findall(review)
    if len(m) == 0:
        m.append("(0)")
    review_list.append(m[0][1:-1])

item.review = review_list

## 카테고리 결측값 기타로, 쌍따옴표 들어가는 경우 제거

In [27]:
item.fillna("기타", inplace=True)

In [34]:
item.category = item.category.apply(lambda x: x.split('|')[0].strip('"'))

# house.csv

In [14]:
house["size"] = house["size"].fillna("0평").apply(lambda x:int(x.rstrip("평대이상하미만 ")))

# csv 파일 추출

In [15]:
item.to_csv("item.tsv", sep="\t", index=False)
house.to_csv("house.tsv", sep="\t", index=False)